In [2]:
# https://www.kaggle.com/pratyushpatnaik/breast-cancer-detector-0-86-accuracy

# Import Keras Layers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Input
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import keras_preprocessing.image as IMAGE


#Import Train Test Split
from sklearn.model_selection import train_test_split

#Import Basic Libraries
import random
import numpy as np
import pandas as pd
import os
import itertools
from PIL import Image
import cv2
import glob
import time
from tqdm import tqdm

In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [33]:
class Classifier(Sequential):
    def __init__(self, activation = 'relu'):
        super().__init__()

        self.input_path = r'input'
        self.activation = activation

        self.img_rows = 50
        self.img_cols = 50

        self.createArchitecture()

    def createArchitecture(self):
        # Block 1
        self.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(self.img_rows,self.img_cols,3)))
        self.add(Activation('elu'))
        self.add(BatchNormalization())
        self.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(self.img_rows,self.img_cols,3)))
        self.add(Activation('elu'))
        self.add(BatchNormalization())
        self.add(MaxPooling2D(pool_size=(2,2)))
        self.add(Dropout(0.3))

        # Block 2
        self.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
        self.add(Activation('elu'))
        self.add(BatchNormalization())
        self.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
        self.add(Activation('elu'))
        self.add(BatchNormalization())
        self.add(MaxPooling2D(pool_size=(2,2)))
        self.add(Dropout(0.3))

        # Block 3
        self.add(Flatten())
        self.add(Dense(64,kernel_initializer='he_normal'))
        self.add(Activation('elu'))
        self.add(BatchNormalization())
        self.add(Dropout(0.5))
        self.add(Dense(1,activation='sigmoid'))

        # Compile
        self.compile(loss='categorical_crossentropy',
            optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
            metrics=['accuracy'])

        print(self.summary())

        self.initCallbacks()


    def initCallbacks(self):
        checkpoint = ModelCheckpoint('skin_disease.h5',
            monitor='val_loss',
            mode='min',
            save_best_only=True,
            verbose=1)

        earlystop = EarlyStopping(monitor='val_loss',
            min_delta=0,
            patience=3,
            verbose=1,
            restore_best_weights=True)

        reduce_lr = ReduceLROnPlateau(monitor='val_loss',
            factor=0.2,
            patience=3,
            verbose=1,
            min_delta=0.0001)
        
        self.callbacks = [earlystop,checkpoint,reduce_lr]


    def loadPaths(self):
        print('Loading input paths')
        self.paths = [os.path.join(self.input_path, x, i, img) for x in os.listdir(self.input_path) for i in os.listdir(os.path.join(self.input_path,x)) for img in os.listdir(os.path.join(self.input_path, x, i))]
        print(f'Found {len(self.paths)} images.')

    def loadImages(self):
        # Convert images to array
        print('Loading Images')
        self.data = ((IMAGE.img_to_array(IMAGE.load_img(path, target_size=(self.img_rows, self.img_cols))), int(path[-5])) for path in tqdm(self.paths))
        print('Finished Loading Images')

#         self.x_train,self.x_test,self.y_train,self.y_test = train_test_split(np.stack([i[0] for i in self.data])/255, np.array([i[1] for i in self.data]), random_state=0, test_size=0.3)


    def train(self):
        self.loadPaths()

        self.loadImages()

        history = self.fit(self.x_train, self.y_train, validation_data = (self.x_test, self.y_test), epochs = 100, verbose = 1, batch_size = 256, callbacks=self.callbacks)

In [42]:
c = Classifier()

Model: "classifier_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 50, 50, 32)        896       
_________________________________________________________________
activation_40 (Activation)   (None, 50, 50, 32)        0         
_________________________________________________________________
batch_normalization_40 (Batc (None, 50, 50, 32)        128       
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 50, 50, 32)        9248      
_________________________________________________________________
activation_41 (Activation)   (None, 50, 50, 32)        0         
_________________________________________________________________
batch_normalization_41 (Batc (None, 50, 50, 32)        128       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 25, 25, 32)       

In [43]:
c.loadPaths()
c.loadImages()

Loading input paths
Found 277524 images.
Loading Images



  0%|          | 335/277524 [00:00<01:22, 3346.55it/s]

Finished Loading Images



 29%|██▉       | 80404/277524 [00:14<00:32, 6072.39it/s]


 57%|█████▋    | 159457/277524 [00:27<00:20, 5698.65it/s]


 85%|████████▌ | 237209/277524 [00:41<00:07, 5711.91it/s]


100%|██████████| 277524/277524 [00:48<00:00, 5745.45it/s]


ValueError: Found input variables with inconsistent numbers of samples: [277524, 0]